# 08 — US County-Level Data Collection & Cleaning

Fetch gun homicide rates, population, Gini, drug offenses, poverty, income, gun ownership, and gun law grades
for ~100 of the largest US counties. Merge all datasets on 5-digit FIPS codes and output a combined CSV.

In [1]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
from pathlib import Path
from us_county_data import (
    get_county_gun_homicide_rates,
    get_county_population,
    get_county_gini,
    get_county_drug_offense_rate,
    get_county_poverty_rate,
    get_county_median_income,
    get_county_gun_ownership,
    get_county_giffords_grade,
    get_county_regions,
)

OUTPUT_DIR = Path('../data/processed')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

## Fetch Individual Datasets

In [2]:
# Gun homicide rates (CDC WONDER)
gun_df = get_county_gun_homicide_rates()
print(f"Gun homicide data: {len(gun_df)} counties")
gun_df.head()

Gun homicide data: 101 counties


,fips,county_name,state,gun_homicide_rate
0,06037,Los Angeles County,CA,7.2
1,17031,Cook County,IL,14.5
2,48201,Harris County,TX,12.8
3,04013,Maricopa County,AZ,7.1
4,06073,San Diego County,CA,3.5


In [3]:
# Population (Census ACS 2022)
pop_df = get_county_population()
print(f"Population data: {len(pop_df)} counties")
pop_df.head()

Population data: 101 counties


,fips,county_name,state,population
0,06037,Los Angeles County,CA,9829544
1,17031,Cook County,IL,5173146
2,48201,Harris County,TX,4728030
3,04013,Maricopa County,AZ,4496588
4,06073,San Diego County,CA,3276208


In [4]:
# Gini coefficient (Census ACS 2022)
gini_df = get_county_gini()
print(f"Gini data: {len(gini_df)} counties")
gini_df.head()

Gini data: 101 counties


,fips,county_name,state,gini
0,06037,Los Angeles County,CA,0.507
1,17031,Cook County,IL,0.504
2,48201,Harris County,TX,0.498
3,04013,Maricopa County,AZ,0.460
4,06073,San Diego County,CA,0.465


In [5]:
# Drug offense rates (FBI UCR)
drug_df = get_county_drug_offense_rate()
print(f"Drug offense data: {len(drug_df)} counties")
drug_df.head()

Drug offense data: 101 counties


,fips,county_name,state,drug_offense_rate
0,06037,Los Angeles County,CA,320.0
1,17031,Cook County,IL,580.0
2,48201,Harris County,TX,490.0
3,04013,Maricopa County,AZ,420.0
4,06073,San Diego County,CA,340.0


In [6]:
# Poverty rate (Census ACS 2022)
poverty_df = get_county_poverty_rate()
print(f"Poverty data: {len(poverty_df)} counties")
poverty_df.head()

Poverty data: 101 counties


,fips,county_name,state,poverty_rate
0,06037,Los Angeles County,CA,14.2
1,17031,Cook County,IL,13.1
2,48201,Harris County,TX,15.8
3,04013,Maricopa County,AZ,13.5
4,06073,San Diego County,CA,11.8


In [7]:
# Median household income (Census ACS 2022)
income_df = get_county_median_income()
print(f"Income data: {len(income_df)} counties")
income_df.head()

Income data: 101 counties


,fips,county_name,state,median_income
0,06037,Los Angeles County,CA,75235
1,17031,Cook County,IL,72231
2,48201,Harris County,TX,63802
3,04013,Maricopa County,AZ,72850
4,06073,San Diego County,CA,88240


In [8]:
# Gun ownership % (RAND state-level proxy)
own_df = get_county_gun_ownership()
print(f"Gun ownership data: {len(own_df)} counties")
own_df.head()

Gun ownership data: 101 counties


,fips,county_name,state,gun_ownership_pct
0,06037,Los Angeles County,CA,28.3
1,17031,Cook County,IL,27.8
2,48201,Harris County,TX,45.7
3,04013,Maricopa County,AZ,46.3
4,06073,San Diego County,CA,28.3


In [9]:
# Giffords gun law grades (state-level proxy)
giffords_df = get_county_giffords_grade()
print(f"Giffords grade data: {len(giffords_df)} counties")
giffords_df.head()

Giffords grade data: 101 counties


,fips,county_name,state,giffords_grade,giffords_numeric
0,06037,Los Angeles County,CA,A,11
1,17031,Cook County,IL,A,11
2,48201,Harris County,TX,F,0
3,04013,Maricopa County,AZ,F,0
4,06073,San Diego County,CA,A,11


## Merge All Datasets

In [10]:
# Start with gun homicide as the base
merged = gun_df.copy()

# Merge population
merged = merged.merge(pop_df[['fips', 'population']], on='fips', how='left')

# Merge Gini
merged = merged.merge(gini_df[['fips', 'gini']], on='fips', how='left')

# Merge drug offenses
merged = merged.merge(drug_df[['fips', 'drug_offense_rate']], on='fips', how='left')

# Merge poverty rate
merged = merged.merge(poverty_df[['fips', 'poverty_rate']], on='fips', how='left')

# Merge median income
merged = merged.merge(income_df[['fips', 'median_income']], on='fips', how='left')

# Merge gun ownership (state proxy)
merged = merged.merge(own_df[['fips', 'gun_ownership_pct']], on='fips', how='left')

# Merge Giffords grade (state proxy)
merged = merged.merge(
    giffords_df[['fips', 'giffords_grade', 'giffords_numeric']],
    on='fips', how='left'
)

# Add region
regions = get_county_regions()
merged = merged.merge(regions, on='fips', how='left')

print(f"Merged dataset: {len(merged)} counties")
merged.head(10)

Merged dataset: 101 counties


,fips,county_name,state,gun_homicide_rate,population,gini,drug_offense_rate,poverty_rate,median_income,gun_ownership_pct,giffords_grade,giffords_numeric,region
0,06037,Los Angeles County,CA,7.2,9829544,0.507,320.0,14.2,75235,28.3,A,11,West
1,17031,Cook County,IL,14.5,5173146,0.504,580.0,13.1,72231,27.8,A,11,Midwest
2,48201,Harris County,TX,12.8,4728030,0.498,490.0,15.8,63802,45.7,F,0,South
3,04013,Maricopa County,AZ,7.1,4496588,0.460,420.0,13.5,72850,46.3,F,0,West
4,06073,San Diego County,CA,3.5,3276208,0.465,340.0,11.8,88240,28.3,A,11,West
5,06059,Orange County,CA,1.8,3162245,0.465,260.0,9.1,104419,28.3,A,11,West
6,12086,Miami-Dade County,FL,10.2,2701767,0.512,410.0,16.5,57815,35.3,D,2,South
7,48113,Dallas County,TX,11.4,2613539,0.504,470.0,15.3,62081,45.7,F,0,South
8,36047,Kings County,NY,4.8,2559903,0.512,290.0,19.2,66850,19.9,A,11,Northeast
9,06065,Riverside County,CA,5.6,2418185,0.435,380.0,13.4,73260,28.3,A,11,West


## Data Coverage Summary

In [11]:
import plotly.graph_objects as go

coverage = pd.DataFrame({
    'Metric': ['Gun Homicide Rate', 'Population', 'Gini Coefficient',
               'Drug Offense Rate', 'Poverty Rate', 'Median Income',
               'Gun Ownership %', 'Giffords Grade'],
    'Counties with data': [
        merged['gun_homicide_rate'].notna().sum(),
        merged['population'].notna().sum(),
        merged['gini'].notna().sum(),
        merged['drug_offense_rate'].notna().sum(),
        merged['poverty_rate'].notna().sum(),
        merged['median_income'].notna().sum(),
        merged['gun_ownership_pct'].notna().sum(),
        merged['giffords_numeric'].notna().sum(),
    ]
})
print(f"Counties with ALL metrics: {merged.dropna().shape[0]}")
print()
coverage

Counties with ALL metrics: 101



,Metric,Counties with data
0,Gun Homicide Rate,101
1,Population,101
2,Gini Coefficient,101
3,Drug Offense Rate,101
4,Poverty Rate,101
5,Median Income,101
6,Gun Ownership %,101
7,Giffords Grade,101


In [12]:
fig = go.Figure(data=[
    go.Bar(
        x=coverage['Metric'],
        y=coverage['Counties with data'],
        marker_color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12',
                      '#9b59b6', '#1abc9c', '#e67e22', '#34495e']
    )
])
fig.update_layout(
    title='Data Coverage by Metric (US Counties)',
    yaxis_title='Number of Counties',
    template='plotly_white',
    height=400,
)
fig.show()

## Save Merged Dataset

In [13]:
out_path = OUTPUT_DIR / 'merged_us_county_data.csv'
merged.to_csv(out_path, index=False)
print(f"Saved merged data to {out_path}")
print(f"Shape: {merged.shape}")
merged.describe()

Saved merged data to ../data/processed/merged_us_county_data.csv
Shape: (101, 13)


,gun_homicide_rate,population,gini,drug_offense_rate,poverty_rate,median_income,gun_ownership_pct,giffords_numeric
count,101.000000,1.010000e+02,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000
mean,8.297030,1.244821e+06,0.469515,397.524752,13.940594,70647.306931,39.711881,4.960396
std,6.927748,1.272756e+06,0.039671,119.602725,4.976247,20091.026633,12.617974,4.855761
min,0.700000,9.950000e+04,0.393000,150.000000,5.100000,39820.000000,8.100000,0.000000
25%,3.400000,5.707190e+05,0.441000,310.000000,10.200000,57350.000000,28.300000,0.000000
50%,6.200000,9.211300e+05,0.469000,390.000000,13.500000,67850.000000,44.400000,2.000000
75%,11.400000,1.459762e+06,0.496000,480.000000,16.500000,76215.000000,47.900000,11.000000
max,34.200000,9.829544e+06,0.600000,720.000000,28.600000,140258.000000,66.300000,11.000000
